In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.0'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [736 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,567 kB]
Get:13 http://archive.ubuntu.c

In [25]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType
spark = SparkSession.builder.appName("Recent Cohorts Data").getOrCreate()
print(spark.sparkContext.appName)

Recent Cohorts Data


In [3]:
# Create the file path for the cohort data file
csv_file_path = "/content/drive/MyDrive/Project_4/data_csv_files/Most-Recent-Cohorts-Institution.csv"

# Read the csv file and show the dataframe
cohorts_df = spark.read.csv(csv_file_path, header=True,
                            inferSchema=True
                            )
cohorts_df.show()

+------+-------+------+--------------------+--------------+------+----------+--------------------+--------------------+--------------------+-------+----+----+---------+-------+-------+-------+-------+------+------+-------+---------+----------+-------+--------+--------+----+---+-----+------+-------+---+-----+-------+---------+--------+--------+------------+-------+-------+-------+-------+-------+-------+--------+--------+--------+-------+-------+-------+-------+-------+-------+-------+-------+--------+--------+--------+--------+-------+-----------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------------+-----+----+----------+----------+---------+----------+---------+---------+---------+--------+---------+------------+------------+--------+------------+------------+----

In [4]:
cohorts_df.dtypes

[('UNITID', 'int'),
 ('OPEID', 'string'),
 ('OPEID6', 'string'),
 ('INSTNM', 'string'),
 ('CITY', 'string'),
 ('STABBR', 'string'),
 ('ZIP', 'string'),
 ('ACCREDAGENCY', 'string'),
 ('INSTURL', 'string'),
 ('NPCURL', 'string'),
 ('SCH_DEG', 'string'),
 ('HCM2', 'int'),
 ('MAIN', 'int'),
 ('NUMBRANCH', 'int'),
 ('PREDDEG', 'int'),
 ('HIGHDEG', 'int'),
 ('CONTROL', 'int'),
 ('ST_FIPS', 'int'),
 ('REGION', 'int'),
 ('LOCALE', 'string'),
 ('LOCALE2', 'string'),
 ('LATITUDE', 'string'),
 ('LONGITUDE', 'string'),
 ('CCBASIC', 'string'),
 ('CCUGPROF', 'string'),
 ('CCSIZSET', 'string'),
 ('HBCU', 'string'),
 ('PBI', 'string'),
 ('ANNHI', 'string'),
 ('TRIBAL', 'string'),
 ('AANAPII', 'string'),
 ('HSI', 'string'),
 ('NANTI', 'string'),
 ('MENONLY', 'string'),
 ('WOMENONLY', 'string'),
 ('RELAFFIL', 'string'),
 ('ADM_RATE', 'string'),
 ('ADM_RATE_ALL', 'string'),
 ('SATVR25', 'string'),
 ('SATVR75', 'string'),
 ('SATMT25', 'string'),
 ('SATMT75', 'string'),
 ('SATWR25', 'string'),
 ('SATWR75',

## Create Lists of Columns with >50% Null and Privacy Suppressed Data

In [5]:
# Count the rows and create a list to store columns with nulls in over 50% of cells
rows = cohorts_df.count()
null_list = []

# Find the count of "NULL" for each column
for col_name in cohorts_df.columns:
  null_count = cohorts_df.filter(col(col_name) == "NULL").count()
  # Create a percentage variable to sort into the list
  percentage = (null_count / rows) * 100
  if percentage > 50.0:
    null_list.append(col_name)
  print(f"{col_name}: {null_count}")

UNITID: 0
OPEID: 20
OPEID6: 20
INSTNM: 0
CITY: 0
STABBR: 0
ZIP: 0
ACCREDAGENCY: 180
INSTURL: 17
NPCURL: 540
SCH_DEG: 366
HCM2: 0
MAIN: 0
NUMBRANCH: 0
PREDDEG: 0
HIGHDEG: 0
CONTROL: 0
ST_FIPS: 0
REGION: 0
LOCALE: 497
LOCALE2: 6543
LATITUDE: 497
LONGITUDE: 497
CCBASIC: 497
CCUGPROF: 497
CCSIZSET: 497
HBCU: 499
PBI: 499
ANNHI: 499
TRIBAL: 499
AANAPII: 499
HSI: 499
NANTI: 499
MENONLY: 497
WOMENONLY: 497
RELAFFIL: 5654
ADM_RATE: 4586
ADM_RATE_ALL: 4319
SATVR25: 5561
SATVR75: 5561
SATMT25: 5561
SATMT75: 5561
SATWR25: 5841
SATWR75: 5841
SATVRMID: 5561
SATMTMID: 5561
SATWRMID: 5841
ACTCM25: 5551
ACTCM75: 5551
ACTEN25: 5608
ACTEN75: 5608
ACTMT25: 5608
ACTMT75: 5608
ACTWR25: 6243
ACTWR75: 6243
ACTCMMID: 5551
ACTENMID: 5608
ACTMTMID: 5608
ACTWRMID: 6243
SAT_AVG: 5507
SAT_AVG_ALL: 5371
PCIP01: 778
PCIP03: 778
PCIP04: 778
PCIP05: 778
PCIP09: 778
PCIP10: 778
PCIP11: 778
PCIP12: 778
PCIP13: 778
PCIP14: 778
PCIP15: 778
PCIP16: 778
PCIP19: 778
PCIP22: 778
PCIP23: 778
PCIP24: 778
PCIP25: 778
PCIP26: 778

In [6]:
print(null_list)

['LOCALE2', 'RELAFFIL', 'ADM_RATE', 'ADM_RATE_ALL', 'SATVR25', 'SATVR75', 'SATMT25', 'SATMT75', 'SATWR25', 'SATWR75', 'SATVRMID', 'SATMTMID', 'SATWRMID', 'ACTCM25', 'ACTCM75', 'ACTEN25', 'ACTEN75', 'ACTMT25', 'ACTMT75', 'ACTWR25', 'ACTWR75', 'ACTCMMID', 'ACTENMID', 'ACTMTMID', 'ACTWRMID', 'SAT_AVG', 'SAT_AVG_ALL', 'UG', 'UGDS_WHITENH', 'UGDS_BLACKNH', 'UGDS_API', 'UGDS_AIANOLD', 'UGDS_HISPOLD', 'UG_NRA', 'UG_UNKN', 'UG_WHITENH', 'UG_BLACKNH', 'UG_API', 'UG_AIANOLD', 'UG_HISPOLD', 'PPTUG_EF2', 'NPT4_PUB', 'NPT4_PROG', 'NPT4_OTHER', 'NPT41_PUB', 'NPT42_PUB', 'NPT43_PUB', 'NPT44_PUB', 'NPT45_PUB', 'NPT42_PRIV', 'NPT43_PRIV', 'NPT44_PRIV', 'NPT45_PRIV', 'NPT41_PROG', 'NPT42_PROG', 'NPT43_PROG', 'NPT44_PROG', 'NPT45_PROG', 'NPT41_OTHER', 'NPT42_OTHER', 'NPT43_OTHER', 'NPT44_OTHER', 'NPT45_OTHER', 'NPT4_048_PUB', 'NPT4_048_PROG', 'NPT4_048_OTHER', 'NPT4_3075_PUB', 'NPT4_3075_PRIV', 'NPT4_75UP_PUB', 'NPT4_75UP_PRIV', 'NPT4_3075_PROG', 'NPT4_3075_OTHER', 'NPT4_75UP_PROG', 'NPT4_75UP_OTHER', 'N

In [7]:
# Count the rows and create a list to store columns with nulls in over 50% of cells
privacy_list = []

# Find the count of "NULL" for each column
for col_name in cohorts_df.columns:
  privacy_count = cohorts_df.filter(col(col_name) == "PrivacySuppressed").count()
  # Create a percentage variable to sort into the list
  percentage = (privacy_count / rows) * 100
  if percentage > 50.0:
    privacy_list.append(col_name)
  print(f"{col_name}: {privacy_count}")

UNITID: 0
OPEID: 0
OPEID6: 0
INSTNM: 0
CITY: 0
STABBR: 0
ZIP: 0
ACCREDAGENCY: 0
INSTURL: 0
NPCURL: 0
SCH_DEG: 0
HCM2: 0
MAIN: 0
NUMBRANCH: 0
PREDDEG: 0
HIGHDEG: 0
CONTROL: 0
ST_FIPS: 0
REGION: 0
LOCALE: 0
LOCALE2: 0
LATITUDE: 0
LONGITUDE: 0
CCBASIC: 0
CCUGPROF: 0
CCSIZSET: 0
HBCU: 0
PBI: 0
ANNHI: 0
TRIBAL: 0
AANAPII: 0
HSI: 0
NANTI: 0
MENONLY: 0
WOMENONLY: 0
RELAFFIL: 0
ADM_RATE: 0
ADM_RATE_ALL: 0
SATVR25: 0
SATVR75: 0
SATMT25: 0
SATMT75: 0
SATWR25: 0
SATWR75: 0
SATVRMID: 0
SATMTMID: 0
SATWRMID: 0
ACTCM25: 0
ACTCM75: 0
ACTEN25: 0
ACTEN75: 0
ACTMT25: 0
ACTMT75: 0
ACTWR25: 0
ACTWR75: 0
ACTCMMID: 0
ACTENMID: 0
ACTMTMID: 0
ACTWRMID: 0
SAT_AVG: 0
SAT_AVG_ALL: 0
PCIP01: 0
PCIP03: 0
PCIP04: 0
PCIP05: 0
PCIP09: 0
PCIP10: 0
PCIP11: 0
PCIP12: 0
PCIP13: 0
PCIP14: 0
PCIP15: 0
PCIP16: 0
PCIP19: 0
PCIP22: 0
PCIP23: 0
PCIP24: 0
PCIP25: 0
PCIP26: 0
PCIP27: 0
PCIP29: 0
PCIP30: 0
PCIP31: 0
PCIP38: 0
PCIP39: 0
PCIP40: 0
PCIP41: 0
PCIP42: 0
PCIP43: 0
PCIP44: 0
PCIP45: 0
PCIP46: 0
PCIP47: 0
PCIP48: 0
PCIP4

In [11]:
# Export both the lists to the correct folder for checkpoint/future use
file_path = "/content/drive/MyDrive/Project_4/null_privacy_columns/"

# Define a dictionary for the files
export_lists = {'null_columns': null_list, 'privacy_suppressed_columns': privacy_list}

for name, lst in export_lists.items():
  file_name = f"{name}.txt"
  with open(file_path + file_name, 'w') as text_file:
    for item in lst:
      text_file.write(f"{item}\n")

  print(f"'{name}' has been successfully exported to '{file_path + file_name}")

'null_columns' has been successfully exported to '/content/drive/MyDrive/Project_4/null_privacy_columns/null_columns.txt
'privacy_suppressed_columns' has been successfully exported to '/content/drive/MyDrive/Project_4/null_privacy_columns/privacy_suppressed_columns.txt


In [20]:
# Combine the lists together for easier removal from the dataframe
null_priv_list = null_list + privacy_list

In [18]:
# Check the dimensions of the original dataframe
original_cohort = (cohorts_df.count(), len(cohorts_df.columns))
print(original_cohort)

(6543, 3024)


In [21]:
# Create a new dataframe excluding the above columns
cleaned_cohorts_df = cohorts_df.select([column for column in cohorts_df.columns if column not in null_priv_list])
print(cleaned_cohorts_df.count(), len(cleaned_cohorts_df.columns))
cleaned_cohorts_df.show()

6543 1256
+------+-------+------+--------------------+--------------+------+----------+--------------------+--------------------+--------------------+-------+----+----+---------+-------+-------+-------+-------+------+------+---------+----------+-------+--------+--------+----+---+-----+------+-------+---+-----+-------+---------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------------+-----+----------+----------+---------+----------+---------+---------+---------+--------+---------+--------+--------+---------+----------+-------------+---------+----------+----------+----------+----------+----------+--------+-------------+--------------+-------+--------+---------+------+-------+-------+-----------+----------+--------+-------+-----+-----+----------------+-------------------